In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import NMF
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from sklearn.metrics import mean_squared_error

In [2]:
processed_df = pd.read_pickle("data/processed_recipes.pkl")
processed_df

,name,ingredients,description,instructions,cuisine,course,diet,prep_time,categorized_prep_time,cleaned_ingredients,allergens,processed_name,processed_instructions,processed_description,allergen_type,processed_ingredients,combined_all_contexts,combined_name_ingredients
0,Doddapatre Tambuli Recipe (Karpooravalli Thayi...,"4 cups Indian borage (Doddapatre) , cleaned an...",Doddapatre Tambuli Recipe is a popular healing...,"To begin making the Doddapatre Tambuli, first ...",Indian,Side Dish,Vegetarian,30.0,~30 min,"[indian borage doddapatre, curd dahi yogurt, c...",[yogurt],doddapatre tambuli karpooravalli thayir pachadi,doddapatre tambuli doddapatre leave dry towel ...,doddapatre tambuli popular healing cold yoghur...,[Dairy],indian borage doddapatre curd dahi yogurt cumi...,doddapatre tambuli karpooravalli thayir pachad...,doddapatre tambuli karpooravalli thayir pachad...
1,Fish Tandoori Recipe,"1 Fish , nicely washed with bones on (I used P...",Have you ever tried out fish tandoori? Trust m...,"To begin making the Fish Tandoori recipe, keep...",North Indian,Appetizer,High Protein Non Vegetarian,80.0,~1 hour 30 min,"[fish, curd dahi yogurt, onion, green chilli, ...",[fish],fish tandoori,fish tandoori head fin inside fish many horizo...,fish tandoori delicious chicken tandoori tando...,[Fish and Shellfish],fish curd dahi yogurt onion green chilli tando...,fish tandoori fish curd dahi yogurt onion gree...,fish tandoori fish curd dahi yogurt onion gree...
2,Arbi Shimla Mirch Sabzi Recipe – Colocasia Cap...,300 grams Colocasia root (Arbi) 1 Green Bell ...,Arbi Shimla Mirch Sabzi Recipe is a variation ...,To begin making the Arbi Shimla Mirch Sabzi Re...,North Indian,Main Course,No Onion No Garlic (Sattvic),55.0,~1 hour,"[colocasia root arbi, green bell pepper capsic...",[peanut],arbi shimla mirch sabzi colocasia capsicum sabzi,arbi shimla mirch sabzi arbi pressure pressure...,arbi shimla mirch sabzi variation aloo capsicu...,"[Tree Nuts and Peanuts, Legumes and Seeds]",colocasia root arbi green bell pepper capsicum...,arbi shimla mirch sabzi colocasia capsicum sab...,arbi shimla mirch sabzi colocasia capsicum sab...
3,Ambur Style Brinjal Curry Recipe,5 Brinjal (Baingan / Eggplant) 1 Onion 2 Tom...,Brinjal Curry is prepared in different ways in...,To begin making the Ambur Style Brinjal Curry ...,Indian,Lunch,Vegetarian,20.0,~30 min,"[brinjal baingan eggplant, onion, ginger garli...","[sesame, peanut, mustard]",ambur style brinjal curry,ambur style brinjal curry oil heavy bottomed o...,brinjal curry different way different region s...,"[Legumes and Seeds, Tree Nuts and Peanuts]",brinjal baingan eggplant onion ginger garlic p...,ambur style brinjal curry brinjal baingan eggp...,ambur style brinjal curry brinjal baingan eggp...
4,Mavinakayi Menasinakai Curry Recipe - Raw Man...,"1 cup Mango (Raw) , diced 2 tablespoons Jagge...",Mangalorean Mavinakayi Menasinakai Curry Recip...,To begin making Mangalorean Mavinakayi Menasin...,Mangalorean,Side Dish,Vegetarian,35.0,~45 min,"[mango raw, jaggery, turmeric powder haldi, ta...",[sesame],mavinakayi menasinakai curry raw mango coconut...,mangalorean mavinakayi menasinakai curry raw m...,mangalorean mavinakayi menasinakai curry raw m...,[Legumes and Seeds],mango raw jaggery turmeric powder haldi tamari...,mavinakayi menasinakai curry raw mango coconut...,mavinakayi menasinakai curry raw mango coconut...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9491,Spinach Oats Vada Recipe,"Spinach Leaves (Palak), Chana dal (Bengal Gram...",Spinach Oats Vada Recipe is a healthy twist to...,"To prepare Spinach Oats Vada Recipe, in a larg...",South Indian,Snack,Vegetarian,50.0,~1 hour,"[spinach leave palak, chana dal bengal gram da...",[oats],spinach oats vada,spinach oats vada large mixing ingredient vada...,spinach oats vada healthy twist vada superfood...,[Grains and Gluten],spinach leave palak chana dal bengal gram dal ...,spinach oats vada spinach leave palak chana da...,sp

Train the recommendation model using NMF model on TF-IDF vectorizer

In [3]:
# Define a function to filter recipes based on user preferences
def get_recommendations_nmf_tfidf(dish_name, num_recommendations=5, ingredients=None, cuisine=None, course=None, diet=None, prep_time=None, allergen_type=None): 

    # Filter recipes based on the provided criteria
    filtered_df = processed_df[processed_df['name'].str.contains(dish_name, case=False, na=False)]
    print ((f"Number of found dishes before filtering : {filtered_df.shape[0]}"))

    if ingredients:
        # Check if all specified ingredients are present in the cleaned_ingredients list
        filtered_df = filtered_df[filtered_df['cleaned_ingredients'].apply(lambda x: all(ing in x for ing in ingredients))]            
    if cuisine:
        filtered_df = filtered_df[filtered_df['cuisine'] == cuisine]
    if course:
        filtered_df = filtered_df[filtered_df['course'] == course]
    if diet:
        filtered_df = filtered_df[filtered_df['diet'] == diet]
    if prep_time:
        filtered_df = filtered_df[filtered_df['categorized_prep_time'] == prep_time]
    if allergen_type:
        # Exclude recipes with allergens in user preference
        filtered_df = filtered_df[~filtered_df['allergen_type'].apply(lambda x: bool(set(x) & set(allergen_type)))]

    if filtered_df.empty:
        print("No recipes found matching the criteria.")
        return None

    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(filtered_df['processed_name'])

    # Ensure all values in tfidf_matrix are non-negative and finite
    tfidf_matrix = np.nan_to_num(tfidf_matrix.toarray(), nan=0.0, posinf=0.0, neginf=0.0)

    # Dynamically determine the best number of topics
    min_topics = 2
    max_topics = min(15, tfidf_matrix.shape[0])  # Limit to the number of rows
    best_num_topics = min_topics
    best_error = float('inf')

    for num_topics in range(min_topics, max_topics + 1):
        nmf_model = NMF(n_components=num_topics, random_state=42)
        nmf_model.fit(tfidf_matrix)
        topic_matrix = nmf_model.transform(tfidf_matrix)
        reconstruction_error = mean_squared_error(tfidf_matrix, np.dot(topic_matrix, nmf_model.components_))
    
        if reconstruction_error < best_error:
            best_error = reconstruction_error
            best_num_topics = num_topics

    print(f"Best number of topics: {best_num_topics}")


    # Train the NMF model with the optimal number of topics    
    nmf_tfidf_model = NMF(n_components=best_num_topics, random_state=42)
    nmf_tfidf_model.fit(tfidf_matrix)
    topic_matrix = nmf_tfidf_model.transform(tfidf_matrix)  # Topic distribution for each recipe

    filtered_df = filtered_df.copy()

    # Add topic distributions to the filtered DataFrame
    for i in range(best_num_topics):
        filtered_df[f'topic_{i}'] = topic_matrix[:, i]

    # Locate the index of the dish within the filtered DataFrame
    dish_index = filtered_df.reset_index().index[0]  # Get the first match if there are multiple

    # Calculate cosine similarity for the topics
    cosine_similarities = cosine_similarity(topic_matrix)
    
    # Get indices of the most similar recipes
    similar_indices = cosine_similarities[dish_index].argsort()[-num_recommendations-1:-1][::-1]

    # Fetch recommended recipes
    recommended_recipes = filtered_df.iloc[similar_indices].copy()
    recommended_recipes["similarity_score"] = cosine_similarities[dish_index][similar_indices]

    # Scale and format similarity scores to percentages
    recommended_recipes['similarity_score'] = (recommended_recipes['similarity_score'] * 100).round(2).astype(str) + '%'

    return recommended_recipes[['name', 'similarity_score', 'cleaned_ingredients', 'cuisine', 'course', 'diet', 'allergens', 'prep_time']]

Get recipe recommendation using best NMF model on TF-IDF vectorizer

In [4]:
recommended_recipes_nmf_tfidf = get_recommendations_nmf_tfidf("masala dosa") ####ingredients = ["ghee", "idli rice"])  
recommended_recipes_nmf_tfidf

Number of found dishes before filtering : 26
Best number of topics: 13


,name,similarity_score,cleaned_ingredients,cuisine,course,diet,allergens,prep_time
5609,Mysore Masala Dosa Made With Ragi Dosa Batter,100.0%,"[idli rice, ragi seed, white urad dal whole, m...",Karnataka,South Indian Breakfast,Vegetarian,[mustard],60.0
1005,Mushroom Corn Masala Dosa Recipe Using Ragi Id...,0.14%,"[white urad dal whole, methi seed fenugreek se...",Fusion,South Indian Breakfast,Vegetarian,unknown,35.0
6039,Mushroom Corn Masala Dosa Recipe Using Ragi Id...,0.14%,"[white urad dal whole, methi seed fenugreek se...",Fusion,South Indian Breakfast,Vegetarian,[peanut],35.0
2546,Kollu Masala Dosa Recipe - High Protein Horse ...,0.0%,"[rice, horse gram dal kulith, methi seed fenug...",Konkan,South Indian Breakfast,Vegetarian,[mustard],960.0
1007,Kanchipuram Masala Dosa Recipe With Soya Dosa ...,0.0%,"[idli rice, soybean whole soya dal, white urad...",South Indian,South Indian Breakfast,Vegetarian,"[soy, mustard, soya]",390.0


Train the recommendation model using best NMF model on Count vectorizer

In [5]:
# Define a function to filter recipes based on user preferences
def get_recommendations_nmf_count(dish_name, num_recommendations=5, ingredients=None, cuisine=None, course=None, diet=None, prep_time=None, allergen_type=None): 

    # Filter recipes based on the provided criteria
    filtered_df = processed_df[processed_df['name'].str.contains(dish_name, case=False, na=False)]
    print ((f"Number of found dishes before filtering : {filtered_df.shape[0]}"))

    if ingredients:
        # Check if all specified ingredients are present in the cleaned_ingredients list
        filtered_df = filtered_df[filtered_df['cleaned_ingredients'].apply(lambda x: all(ing in x for ing in ingredients))]  
    if cuisine:
        filtered_df = filtered_df[filtered_df['cuisine'] == cuisine]
    if course:
        filtered_df = filtered_df[filtered_df['course'] == course]
    if diet:
        filtered_df = filtered_df[filtered_df['diet'] == diet]
    if prep_time:
        filtered_df = filtered_df[filtered_df['categorized_prep_time'] == prep_time]
    if allergen_type:
        # Exclude recipes with allergens in user preference
        filtered_df = filtered_df[~filtered_df['allergen_type'].apply(lambda x: bool(set(x) & set(allergen_type)))]

    if filtered_df.empty:
        print("No recipes found matching the criteria.")
        return None

    count_vectorizer = CountVectorizer()
    count_matrix = count_vectorizer.fit_transform(filtered_df['processed_name'])

    # Ensure all values in tfidf_matrix are non-negative and finite
    count_matrix = np.nan_to_num(count_matrix.toarray(), nan=0.0, posinf=0.0, neginf=0.0)

    # Dynamically determine the best number of topics
    min_topics = 2
    max_topics = min(15, count_matrix.shape[0])  # Limit to the number of rows
    best_num_topics = min_topics
    best_error = float('inf')

    for num_topics in range(min_topics, max_topics + 1):
        nmf_model = NMF(n_components=num_topics, random_state=42)
        nmf_model.fit(count_matrix)
        topic_matrix = nmf_model.transform(count_matrix)
        reconstruction_error = mean_squared_error(count_matrix, np.dot(topic_matrix, nmf_model.components_))
    
        if reconstruction_error < best_error:
            best_error = reconstruction_error
            best_num_topics = num_topics

    print(f"Best number of topics: {best_num_topics}")


    # Train the NMF model with the optimal number of topics    
    nmf_count_model = NMF(n_components=best_num_topics, random_state=42)
    nmf_count_model.fit(count_matrix)
    topic_matrix = nmf_count_model.transform(count_matrix)  # Topic distribution for each recipe

    filtered_df = filtered_df.copy()

    # Add topic distributions to the filtered DataFrame
    for i in range(best_num_topics):
        filtered_df[f'topic_{i}'] = topic_matrix[:, i]

    # Locate the index of the dish within the filtered DataFrame
    dish_index = filtered_df.reset_index().index[0]  # Get the first match if there are multiple

    # Calculate cosine similarity for the topics
    cosine_similarities = cosine_similarity(topic_matrix)
    
    # Get indices of the most similar recipes
    similar_indices = cosine_similarities[dish_index].argsort()[-num_recommendations-1:-1][::-1]

    # Fetch recommended recipes
    recommended_recipes = filtered_df.iloc[similar_indices].copy()
    recommended_recipes["similarity_score"] = cosine_similarities[dish_index][similar_indices]

    # Scale and format similarity scores to percentages
    recommended_recipes['similarity_score'] = (recommended_recipes['similarity_score'] * 100).round(2).astype(str) + '%'

    return recommended_recipes[['name', 'similarity_score', 'cleaned_ingredients', 'cuisine', 'course', 'diet', 'allergens', 'prep_time']]

Get recipe recommendation using best NMF model on Count vectorizer

In [6]:
recommended_recipes_nmf_count = get_recommendations_nmf_count("masala dosa")  
recommended_recipes_nmf_count

Number of found dishes before filtering : 26
Best number of topics: 15


c:\Users\rogha\anaconda3\envs\Tasty_AI\lib\site-packages\sklearn\decomposition\_nmf.py:1759: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(
c:\Users\rogha\anaconda3\envs\Tasty_AI\lib\site-packages\sklearn\decomposition\_nmf.py:1759: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


,name,similarity_score,cleaned_ingredients,cuisine,course,diet,allergens,prep_time
5609,Mysore Masala Dosa Made With Ragi Dosa Batter,100.0%,"[idli rice, ragi seed, white urad dal whole, m...",Karnataka,South Indian Breakfast,Vegetarian,[mustard],60.0
6428,Egg Roast Masala Dosa Recipe - With Moong Dal ...,2.74%,"[yellow moong dal split, idli rice, methi seed...",South Indian,South Indian Breakfast,Eggetarian,"[mustard, egg]",30.0
1238,Egg Roast Masala Dosa Recipe - With Moong Dal ...,2.74%,"[yellow moong dal split, idli rice, methi seed...",South Indian,South Indian Breakfast,Eggetarian,"[mustard, egg]",30.0
6041,Kanchipuram Masala Dosa Recipe With Soya Dosa ...,2.11%,"[idli rice, soybean whole soya dal, white urad...",South Indian,South Indian Breakfast,Vegetarian,"[soy, mustard, soya]",390.0
1007,Kanchipuram Masala Dosa Recipe With Soya Dosa ...,2.11%,"[idli rice, soybean whole soya dal, white urad...",South Indian,South Indian Breakfast,Vegetarian,"[soy, mustard, soya]",390.0


Train the recommendation model using SVD model on TF-IDF vectorizer

In [7]:
# Define a function to filter recipes based on user preferences
def get_recommendations_svd_tfidf(dish_name, num_recommendations=5, ingredients=None, cuisine=None, course=None, diet=None, prep_time=None, allergen_type=None): 

    # Filter recipes based on the provided criteria
    filtered_df = processed_df[processed_df['name'].str.contains(dish_name, case=False, na=False)]
    print ((f"Number of found dishes before filtering : {filtered_df.shape[0]}"))

    if ingredients:
        # Check if all specified ingredients are present in the cleaned_ingredients list
        filtered_df = filtered_df[filtered_df['cleaned_ingredients'].apply(lambda x: all(ing in x for ing in ingredients))]  
    if cuisine:
        filtered_df = filtered_df[filtered_df['cuisine'] == cuisine]
    if course:
        filtered_df = filtered_df[filtered_df['course'] == course]
    if diet:
        filtered_df = filtered_df[filtered_df['diet'] == diet]
    if prep_time:
        filtered_df = filtered_df[filtered_df['categorized_prep_time'] == prep_time]
    if allergen_type:
        # Exclude recipes with allergens in user preference
        filtered_df = filtered_df[~filtered_df['allergen_type'].apply(lambda x: bool(set(x) & set(allergen_type)))]

    if filtered_df.empty:
        print("No recipes found matching the criteria.")
        return None

    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(filtered_df['processed_name'])

    # Ensure all values in tfidf_matrix are non-negative and finite
    tfidf_matrix = np.nan_to_num(tfidf_matrix.toarray(), nan=0.0, posinf=0.0, neginf=0.0)

    # Dynamically determine the best number of topics
    min_topics = 2
    max_topics = min(15, tfidf_matrix.shape[0])  # Limit to the number of rows
    best_num_topics = min_topics
    best_error = float('inf')

    for num_topics in range(min_topics, max_topics + 1):
        svd_model = TruncatedSVD(n_components=num_topics, random_state=42)
        svd_model.fit(tfidf_matrix)
        topic_matrix = svd_model.transform(tfidf_matrix)
        reconstruction_error = mean_squared_error(tfidf_matrix, np.dot(topic_matrix, svd_model.components_))
    
        if reconstruction_error < best_error:
            best_error = reconstruction_error
            best_num_topics = num_topics

    print(f"Best number of topics: {best_num_topics}")


    # Train the NMF model with the optimal number of topics    
    svd_tfidf_model = TruncatedSVD(n_components=best_num_topics, random_state=42)
    svd_tfidf_model.fit(tfidf_matrix)
    topic_matrix = svd_tfidf_model.transform(tfidf_matrix)  # Topic distribution for each recipe

    filtered_df = filtered_df.copy()

    # Add topic distributions to the filtered DataFrame
    for i in range(best_num_topics):
        filtered_df[f'topic_{i}'] = topic_matrix[:, i]

    # Locate the index of the dish within the filtered DataFrame
    dish_index = filtered_df.reset_index().index[0]  # Get the first match if there are multiple

    # Calculate cosine similarity for the topics
    cosine_similarities = cosine_similarity(topic_matrix)
    
    # Get indices of the most similar recipes
    similar_indices = cosine_similarities[dish_index].argsort()[-num_recommendations-1:-1][::-1]

    # Fetch recommended recipes
    recommended_recipes = filtered_df.iloc[similar_indices].copy()
    recommended_recipes["similarity_score"] = cosine_similarities[dish_index][similar_indices]

    # Scale and format similarity scores to percentages
    recommended_recipes['similarity_score'] = (recommended_recipes['similarity_score'] * 100).round(2).astype(str) + '%'

    return recommended_recipes[['name', 'similarity_score', 'cleaned_ingredients', 'cuisine', 'course', 'diet', 'allergens', 'prep_time']]

Get recipe recommendation using best SVD model on TF-IDF vectorizer

In [8]:
recommended_recipes_svd_tfidf = get_recommendations_svd_tfidf("masala dosa")  
recommended_recipes_svd_tfidf

Number of found dishes before filtering : 26
Best number of topics: 14


,name,similarity_score,cleaned_ingredients,cuisine,course,diet,allergens,prep_time
5609,Mysore Masala Dosa Made With Ragi Dosa Batter,100.0%,"[idli rice, ragi seed, white urad dal whole, m...",Karnataka,South Indian Breakfast,Vegetarian,[mustard],60.0
1005,Mushroom Corn Masala Dosa Recipe Using Ragi Id...,48.73%,"[white urad dal whole, methi seed fenugreek se...",Fusion,South Indian Breakfast,Vegetarian,unknown,35.0
6039,Mushroom Corn Masala Dosa Recipe Using Ragi Id...,48.73%,"[white urad dal whole, methi seed fenugreek se...",Fusion,South Indian Breakfast,Vegetarian,[peanut],35.0
6041,Kanchipuram Masala Dosa Recipe With Soya Dosa ...,31.28%,"[idli rice, soybean whole soya dal, white urad...",South Indian,South Indian Breakfast,Vegetarian,"[soy, mustard, soya]",390.0
1007,Kanchipuram Masala Dosa Recipe With Soya Dosa ...,31.28%,"[idli rice, soybean whole soya dal, white urad...",South Indian,South Indian Breakfast,Vegetarian,"[soy, mustard, soya]",390.0


Train the recommendation model using best SVD model on Count vectorizer

In [9]:
# Define a function to filter recipes based on user preferences
def get_recommendations_svd_count(dish_name, num_recommendations=5, ingredients=None, cuisine=None, course=None, diet=None, prep_time=None, allergen_type=None): 

    # Filter recipes based on the provided criteria
    filtered_df = processed_df[processed_df['name'].str.contains(dish_name, case=False, na=False)]
    print ((f"Number of found dishes before filtering : {filtered_df.shape[0]}"))

    if ingredients:
        # Check if all specified ingredients are present in the cleaned_ingredients list
        filtered_df = filtered_df[filtered_df['cleaned_ingredients'].apply(lambda x: all(ing in x for ing in ingredients))] 
    if cuisine:
        filtered_df = filtered_df[filtered_df['cuisine'] == cuisine]
    if course:
        filtered_df = filtered_df[filtered_df['course'] == course]
    if diet:
        filtered_df = filtered_df[filtered_df['diet'] == diet]
    if prep_time:
        filtered_df = filtered_df[filtered_df['categorized_prep_time'] == prep_time]
    if allergen_type:
        # Exclude recipes with allergens in user preference
        filtered_df = filtered_df[~filtered_df['allergen_type'].apply(lambda x: bool(set(x) & set(allergen_type)))]

    if filtered_df.empty:
        print("No recipes found matching the criteria.")
        return None

    count_vectorizer = CountVectorizer()
    count_matrix = count_vectorizer.fit_transform(filtered_df['processed_name'])

    # Ensure all values in count_matrix are non-negative and finite
    count_matrix = np.nan_to_num(count_matrix.toarray(), nan=0.0, posinf=0.0, neginf=0.0)

    # Dynamically determine the best number of topics
    min_topics = 2
    max_topics = min(15, count_matrix.shape[0])  # Limit to the number of rows
    best_num_topics = min_topics
    best_error = float('inf')

    for num_topics in range(min_topics, max_topics + 1):
        svd_model = TruncatedSVD(n_components=num_topics, random_state=42)
        svd_model.fit(count_matrix)
        topic_matrix = svd_model.transform(count_matrix)
        reconstruction_error = mean_squared_error(count_matrix, np.dot(topic_matrix, svd_model.components_))
    
        if reconstruction_error < best_error:
            best_error = reconstruction_error
            best_num_topics = num_topics

    print(f"Best number of topics: {best_num_topics}")


    # Train the NMF model with the optimal number of topics    
    svd_count_model = TruncatedSVD(n_components=best_num_topics, random_state=42)
    svd_count_model.fit(count_matrix)
    topic_matrix = svd_count_model.transform(count_matrix)  # Topic distribution for each recipe

    filtered_df = filtered_df.copy()

    # Add topic distributions to the filtered DataFrame
    for i in range(best_num_topics):
        filtered_df[f'topic_{i}'] = topic_matrix[:, i]

    # Locate the index of the dish within the filtered DataFrame
    dish_index = filtered_df.reset_index().index[0]  # Get the first match if there are multiple

    # Calculate cosine similarity for the topics
    cosine_similarities = cosine_similarity(topic_matrix)
    
    # Get indices of the most similar recipes
    similar_indices = cosine_similarities[dish_index].argsort()[-num_recommendations-1:-1][::-1]

    # Fetch recommended recipes
    recommended_recipes = filtered_df.iloc[similar_indices].copy()
    recommended_recipes["similarity_score"] = cosine_similarities[dish_index][similar_indices]

    # Scale and format similarity scores to percentages
    recommended_recipes['similarity_score'] = (recommended_recipes['similarity_score'] * 100).round(2).astype(str) + '%'

    return recommended_recipes[['name', 'similarity_score', 'cleaned_ingredients', 'cuisine', 'course', 'diet', 'allergens', 'prep_time']]

Get recipe recommendation using best SVD model on Count vectorizer

In [10]:
recommended_recipes_svd_count = get_recommendations_svd_count("masala dosa")  
recommended_recipes_svd_count

Number of found dishes before filtering : 26
Best number of topics: 14


,name,similarity_score,cleaned_ingredients,cuisine,course,diet,allergens,prep_time
5609,Mysore Masala Dosa Made With Ragi Dosa Batter,100.0%,"[idli rice, ragi seed, white urad dal whole, m...",Karnataka,South Indian Breakfast,Vegetarian,[mustard],60.0
1005,Mushroom Corn Masala Dosa Recipe Using Ragi Id...,78.26%,"[white urad dal whole, methi seed fenugreek se...",Fusion,South Indian Breakfast,Vegetarian,unknown,35.0
6039,Mushroom Corn Masala Dosa Recipe Using Ragi Id...,78.26%,"[white urad dal whole, methi seed fenugreek se...",Fusion,South Indian Breakfast,Vegetarian,[peanut],35.0
6041,Kanchipuram Masala Dosa Recipe With Soya Dosa ...,75.0%,"[idli rice, soybean whole soya dal, white urad...",South Indian,South Indian Breakfast,Vegetarian,"[soy, mustard, soya]",390.0
1007,Kanchipuram Masala Dosa Recipe With Soya Dosa ...,75.0%,"[idli rice, soybean whole soya dal, white urad...",South Indian,South Indian Breakfast,Vegetarian,"[soy, mustard, soya]",390.0


Search more

In [11]:
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from sklearn.metrics.pairwise import cosine_similarity

def recommend_recipes(dish_name, num_recommendations=5, ingredients=None, cuisine=None, course=None, diet=None, prep_time=None, allergen_type=None): 
    # Filter recipes based on the provided criteria
    filtered_df = processed_df[processed_df['name'].str.contains(dish_name, case=False, na=False)]
    print(f"Number of found dishes before filtering : {filtered_df.shape[0]}")
    if cuisine:
        filtered_df = filtered_df[filtered_df['cuisine'] == cuisine]
    if course:
        filtered_df = filtered_df[filtered_df['course'] == course]
    if diet:
        filtered_df = filtered_df[filtered_df['diet'] == diet]
    if prep_time:
        filtered_df = filtered_df[filtered_df['categorized_prep_time'] == prep_time]
    if allergen_type:
        # Exclude recipes with allergens in user preference
        filtered_df = filtered_df[~filtered_df['allergen_type'].apply(lambda x: bool(set(x) & set(allergen_type)))]

    if filtered_df.empty:
        print("No recipes found matching the criteria.")
        return None

    # TF-IDF Vectorization on 'processed_name'
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(filtered_df['processed_name'])

    # Ensure all values in tfidf_matrix are non-negative and finite
    tfidf_matrix = np.nan_to_num(tfidf_matrix.toarray(), nan=0.0, posinf=0.0, neginf=0.0)

    # Dynamically determine the best number of topics
    min_topics = 2
    max_topics = min(15, tfidf_matrix.shape[0])  # Limit to the number of rows
    best_num_topics = min_topics
    best_error = float('inf')

    for num_topics in range(min_topics, max_topics + 1):
        nmf_model = NMF(n_components=num_topics, random_state=42)
        nmf_model.fit(tfidf_matrix)
        topic_matrix = nmf_model.transform(tfidf_matrix)
        reconstruction_error = mean_squared_error(tfidf_matrix, np.dot(topic_matrix, nmf_model.components_))
    
        if reconstruction_error < best_error:
            best_error = reconstruction_error
            best_num_topics = num_topics

    print(f"Best number of topics: {best_num_topics}")

    # Train the NMF model with the optimal number of topics
    nmf_tfidf_model = NMF(n_components=best_num_topics, random_state=42)
    nmf_tfidf_model.fit(tfidf_matrix)
    topic_matrix = nmf_tfidf_model.transform(tfidf_matrix)  # Topic distribution for each recipe

    filtered_df = filtered_df.copy()

    # Add topic distributions to the filtered DataFrame
    for i in range(best_num_topics):
        filtered_df[f'topic_{i}'] = topic_matrix[:, i]

    # Aggregate the topic distributions for all matched dishes
    aggregate_topic_vector = topic_matrix.mean(axis=0).reshape(1, -1)

    # Calculate cosine similarity against the aggregate topic vector
    cosine_similarities = cosine_similarity(topic_matrix, aggregate_topic_vector).flatten()

    # Get indices of the most similar recipes
    similar_indices = cosine_similarities.argsort()[-num_recommendations:][::-1]

    # Fetch recommended recipes
    recommended_recipes = filtered_df.iloc[similar_indices].copy()
    recommended_recipes["similarity_score"] = cosine_similarities[similar_indices]

    # Scale and format similarity scores to percentages
    recommended_recipes['similarity_score'] = (recommended_recipes['similarity_score'] * 100).round(2).astype(str) + '%'

    return recommended_recipes[['name', 'similarity_score', 'cleaned_ingredients', 'cuisine', 'course', 'diet', 'allergen_type', 'categorized_prep_time']]

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from sklearn.metrics.pairwise import cosine_similarity
from gensim.corpora import Dictionary
from gensim.models import CoherenceModel
import numpy as np

# Function to recommend recipes
def recommend_recipes(dish_name, num_recommendations=5, ingredients=None, cuisine=None, course=None, diet=None, prep_time=None, allergen_type=None):
    """
    Recommend recipes based on dish name and user preferences.
    """
    # Filter recipes based on user preferences
    filtered_df = processed_df[processed_df['name'].str.contains(dish_name, case=False, na=False)]
    print(f"Number of found dishes before filtering: {filtered_df.shape[0]}")

    if cuisine:
        filtered_df = filtered_df[filtered_df['cuisine'] == cuisine]
    if course:
        filtered_df = filtered_df[filtered_df['course'] == course]
    if diet:
        filtered_df = filtered_df[filtered_df['diet'] == diet]
    if prep_time:
        filtered_df = filtered_df[filtered_df['categorized_prep_time'] == prep_time]
    if allergen_type:
        # Exclude recipes with allergens in user preference
        filtered_df = filtered_df[~filtered_df['allergen_type'].apply(lambda x: bool(set(x) & set(allergen_type)))]

    if filtered_df.empty:
        print("No recipes found matching the criteria.")
        return None

    # TF-IDF Vectorization on 'processed_name'
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix_sparse = tfidf_vectorizer.fit_transform(filtered_df['processed_name'])  # Keep sparse matrix
    tfidf_matrix_dense = tfidf_matrix_sparse.toarray()  # For dense operations

    # Determine the best number of topics using coherence score
    min_topics = 2
    max_topics = min(15, filtered_df.shape[0])  # Limit to the number of rows
    best_num_topics = min_topics
    best_coherence_score = float('-inf')

    for num_topics in range(min_topics, max_topics + 1):
        nmf_model = NMF(n_components=num_topics, random_state=42)
        topic_matrix = nmf_model.fit_transform(tfidf_matrix_dense)

        # Calculate coherence score within the loop
        feature_names = tfidf_vectorizer.get_feature_names_out()
        topics_words = [
            [feature_names[i] for i in topic.argsort()[:-10 - 1:-1]] for topic in nmf_model.components_
        ]
        tokenized_docs = [doc.split() for doc in tfidf_vectorizer.inverse_transform(tfidf_matrix_sparse)]
        dictionary = Dictionary(tokenized_docs)
        corpus = [dictionary.doc2bow(doc) for doc in tokenized_docs]

        coherence_model = CoherenceModel(
            topics=topics_words, texts=tokenized_docs, dictionary=dictionary, coherence='c_v'
        )
        coherence_score = coherence_model.get_coherence()

        if coherence_score > best_coherence_score:
            best_coherence_score = coherence_score
            best_num_topics = num_topics

    print(f"Best number of topics: {best_num_topics}, Best coherence score: {best_coherence_score:.4f}")

    # Train the NMF model with the optimal number of topics
    nmf_tfidf_model = NMF(n_components=best_num_topics, random_state=42)
    nmf_tfidf_model.fit(tfidf_matrix_dense)
    topic_matrix = nmf_tfidf_model.transform(tfidf_matrix_dense)

    filtered_df = filtered_df.copy()

    # Add topic distributions to the filtered DataFrame
    for i in range(best_num_topics):
        filtered_df[f'topic_{i}'] = topic_matrix[:, i]

    # Locate the index of the dish within the filtered DataFrame
    dish_index = filtered_df.reset_index().index[0]  # Get the first match if there are multiple

    # Calculate cosine similarity for the topics
    cosine_similarities = cosine_similarity(topic_matrix)

    # Get indices of the most similar recipes
    similar_indices = cosine_similarities[dish_index].argsort()[-num_recommendations-1:-1][::-1]

    # Fetch recommended recipes
    recommended_recipes = filtered_df.iloc[similar_indices].copy()
    recommended_recipes["similarity_score"] = cosine_similarities[dish_index][similar_indices]

    # Scale and format similarity scores to percentages
    recommended_recipes['similarity_score'] = (recommended_recipes['similarity_score'] * 100).round(2).astype(str) + '%'

    return recommended_recipes[['name', 'similarity_score', 'cleaned_ingredients', 'cuisine', 'course', 'diet', 'allergen_type', 'categorized_prep_time']]